In [1]:
# !pip install arxiv
# !pip install -U spacy
# !python -m spacy download en_core_web_sm
# !pip install pdfx
# !pip install owlready2

In [2]:
import arxiv
import urllib
import pdfx
import re

# Téléchargement des PDF sur Arkiv

### Utilisation de la librairie ArXiv pou récupérer le lien des pdf

In [3]:
search = arxiv.Search(
    query = "computer science &  ai",
    # id_list=["1605.08386v1"]
    max_results = 2,
    sort_by = arxiv.SortCriterion.SubmittedDate
)

for result in search.results():
    print ("Titre :", result.title)
    print ("Auteurs :", result.authors)
    print ("Lien :", result.pdf_url)

Titre : Temporal Alignment Networks for Long-term Video
Auteurs : [arxiv.Result.Author('Tengda Han'), arxiv.Result.Author('Weidi Xie'), arxiv.Result.Author('Andrew Zisserman')]
Lien : http://arxiv.org/pdf/2204.02968v1
Titre : Enhanced Direct Speech-to-Speech Translation Using Self-supervised Pre-training and Data Augmentation
Auteurs : [arxiv.Result.Author('Sravya Popuri'), arxiv.Result.Author('Peng-Jen Chen'), arxiv.Result.Author('Changhan Wang'), arxiv.Result.Author('Juan Pino'), arxiv.Result.Author('Yossi Adi'), arxiv.Result.Author('Jiatao Gu'), arxiv.Result.Author('Wei-Ning Hsu'), arxiv.Result.Author('Ann Lee')]
Lien : http://arxiv.org/pdf/2204.02967v1


In [4]:
print(result.pdf_url)

http://arxiv.org/pdf/2204.02967v1


In [5]:
# On met les auteurs sur forme de liste pour faciliter le traitement

for result in search.results():
    author = result.authors
    author_list = [re.sub("[^A-Za-z0-9]","_",str(i)) for i in author]

print (author_list)

['Sravya_Popuri', 'Peng_Jen_Chen', 'Changhan_Wang', 'Juan_Pino', 'Yossi_Adi', 'Jiatao_Gu', 'Wei_Ning_Hsu', 'Ann_Lee']


In [6]:
# Download the pdf & store it

response = urllib.request.urlopen(result.pdf_url)    
file = open("Test" + ".pdf", 'wb')
file.write(response.read())
file.close()

# Leture du PDF

pdf = pdfx.PDFx("Test.pdf")
metadata = pdf.get_metadata()
references_dict = pdf.get_references_as_dict()
text = pdf.get_text()

In [7]:
print (metadata)
# print (references_list)
print (references_dict['url'])
#print (text)

{'CreationDate': 'D:20220407004439Z', 'Creator': 'LaTeX with hyperref', 'ModDate': 'D:20220407004439Z', 'PTEX.Fullbanner': 'This is pdfTeX, Version 3.14159265-2.6-1.40.21 (TeX Live 2020) kpathsea version 6.3.2', 'Producer': 'pdfTeX-1.40.21', 'Trapped': 'False', 'Pages': 7}
['2107.05604', '2112.08352', 'https://github', '2010.11745', '2105.01051', 'https://huggingface.co/facebook/tts_', 'https://huggingface.co/facebook/tts_transformer-en-ljspeech', 'facebookresearch.github.io/speech_translation/', 'https://huggingface.co/facebook/tts_transformer-es-css10', 'https://huggingface.co/jonatasgrosman/wav2vec2-large-xlsr-53-spanish', '2104.06678', 'https://github.com/pytorch/fairseq/blob/main/examples/speech_to_speech/docs/textless_s2st_real_data.md', 'https://huggingface', '2111.07402', '2007.10310', '2012.03411', 'github.com/pytorch/fairseq/blob/', 'https://huggingface.co/jonatasgrosman/', '2104.00355', '2109.03264', '2104.02133', '2102.01192', 'https://github.com/pytorch/fairseq/blob/main/e

### Téléchargement d'une référence afin d'avoir des liens pertinents

In [8]:
print(len(references_dict['url']))
for i in range(len(references_dict['url'])):
    try :
        search_ref = arxiv.Search(id_list=[references_dict['url'][i]])
        for result in search_ref.results():
            print ("Titre :", result.title)
            print ("Auteurs :", result.authors)
            print ("Lien :", result.pdf_url)
        break
    except:
        pass

34
Titre : Direct speech-to-speech translation with discrete units
Auteurs : [arxiv.Result.Author('Ann Lee'), arxiv.Result.Author('Peng-Jen Chen'), arxiv.Result.Author('Changhan Wang'), arxiv.Result.Author('Jiatao Gu'), arxiv.Result.Author('Sravya Popuri'), arxiv.Result.Author('Xutai Ma'), arxiv.Result.Author('Adam Polyak'), arxiv.Result.Author('Yossi Adi'), arxiv.Result.Author('Qing He'), arxiv.Result.Author('Yun Tang'), arxiv.Result.Author('Juan Pino'), arxiv.Result.Author('Wei-Ning Hsu')]
Lien : http://arxiv.org/pdf/2107.05604v2


In [9]:
for result in search_ref.results():
    author = result.authors
    author_ref_list = [re.sub("[^A-Za-z0-9]","_",str(i)) for i in author]

print (author_ref_list)

['Ann_Lee', 'Peng_Jen_Chen', 'Changhan_Wang', 'Jiatao_Gu', 'Sravya_Popuri', 'Xutai_Ma', 'Adam_Polyak', 'Yossi_Adi', 'Qing_He', 'Yun_Tang', 'Juan_Pino', 'Wei_Ning_Hsu']


In [10]:
# Download the pdf & store it

response = urllib.request.urlopen(result.pdf_url)    
file = open("Ref" + ".pdf", 'wb')
file.write(response.read())
file.close()

# Leture du PDF

pdf = pdfx.PDFx("Ref.pdf")
metadata_ref = pdf.get_metadata()
references_ref_dict = pdf.get_references_as_dict()
text_ref = pdf.get_text()

### Optimisation du champ de recherche à partir du mot "Références"

In [11]:
def after_references(mypdftext): 
    keyword1 = 'References'
    keyword2 = 'REFERENCES'
    keyword3 = 'R EFERENCES'
    keyword4 = 'Reference'
    keyword5='[1]' 

    if keyword1 in mypdftext :
            before_keyword, keyword, after_keyword = mypdftext.partition(keyword1)
    elif keyword2 in mypdftext :
            before_keyword, keyword, after_keyword = mypdftext.partition(keyword2)
    elif keyword3 in mypdftext :
            before_keyword, keyword, after_keyword = mypdftext.partition(keyword3)
    elif keyword4 in mypdftext :
            before_keyword, keyword, after_keyword = mypdftext.partition(keyword4)
    elif keyword5 in mypdftext :
            before_keyword, keyword, after_keyword = mypdftext.partition(keyword5)
    else:
        after_keyword = mypdftext[:10000]
    return after_keyword

#All references in a variable

references = after_references(text)
references_ref = after_references(text_ref)

# print(references)

# Recherche des entités nommées

In [ ]:
# Commande à executer la première fois car le package ne s'installe pas bien avec le fichier "requirement"

!python -m spacy download en_core_web_sm

In [12]:
import spacy
from spacy.lang.fr.examples import sentences 

def extract(text:str) :
    nlp = spacy.load('en_core_web_sm')
    doc = nlp(text.strip())
    named_entities = []
    
    for i in doc.ents:
        entry = str(i.lemma_).lower()
        text = text.replace(str(i).lower(), "")
        if i.label_ in ["PERSON"]:
            named_entities.append(entry.title().replace(" ", "_").replace("\n","_"))
        named_entities = list(dict.fromkeys(named_entities))
    return named_entities

references_list = extract(references)
references_ref_list = extract(references_ref)

/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.24.3) or chardet (4.0.0) doesn't match a supported version!
  RequestsDependencyWarning)


### Traitement des charactères non reconnus

In [13]:
from xml.sax.saxutils import escape
import sys

def invalid_xml_remove(char):
    """Tracks illegal unicode characters"""
    #http://stackoverflow.com/questions/1707890
    # /fast-way-to-filter-illegal-xml-unicode-chars-in-python
    illegal_unichrs = [ (0x00, 0x08), (0x0B, 0x1F), (0x7F, 0x84), (0x86, 0x9F),
                    (0xD800, 0xDFFF), (0xFDD0, 0xFDDF), (0xFFFE, 0xFFFF),
                    (0x1FFFE, 0x1FFFF), (0x2FFFE, 0x2FFFF), (0x3FFFE, 0x3FFFF),
                    (0x4FFFE, 0x4FFFF), (0x5FFFE, 0x5FFFF), (0x6FFFE, 0x6FFFF),
                    (0x7FFFE, 0x7FFFF), (0x8FFFE, 0x8FFFF), (0x9FFFE, 0x9FFFF),
                    (0xAFFFE, 0xAFFFF), (0xBFFFE, 0xBFFFF), (0xCFFFE, 0xCFFFF),
                    (0xDFFFE, 0xDFFFF), (0xEFFFE, 0xEFFFF), (0xFFFFE, 0xFFFFF),
                    (0x10FFFE, 0x10FFFF) ]

    illegal_ranges = [f"{chr(low)}-{chr(high)}"
                  for (low, high) in illegal_unichrs
                  if low < sys.maxunicode]

    illegal_xml_re = re.compile(f'[{"".join(illegal_ranges)}]')
    if illegal_xml_re.search(char) is not None:
        #Replace with space
        return ''
    else:
        return char

def clean_char(char):
    """
    Function for remove invalid XML characters from
    incoming data.
    """
    #Get rid of the ctrl characters first.
    #http://stackoverflow.com/questions/1833873/python-regex-escape-characters
    char = re.sub('\x1b[^m]*m', '', char)
    #Clean up invalid xml
    char = invalid_xml_remove(char)
    replacements = [
        ('\u201c', '\"'),
        ('\u0141', '\"'),
        ('\u201d', '\"'),
        ("\u001B", ''), #http://www.fileformat.info/info/unicode/char/1b/index.htm
        ("\u0019", ''), #http://www.fileformat.info/info/unicode/char/19/index.htm
        ("\u0016", ''), #http://www.fileformat.info/info/unicode/char/16/index.htm
        ("\u001C", ''), #http://www.fileformat.info/info/unicode/char/1c/index.htm
        ("\u0003", ''), #http://www.utf8-chartable.de/unicode-utf8-table.pl?utf8=0x
        ("\u000C", ''),
        ("\u03b1", ''),
        ("u\u039C", ''),
        ("\u03C3", ''),
        ("\u0141", ''),
        ("\u0308", ''),
        ("\u2032", ''),
        ("\u03b8", '')

    
    ]
    for rep, new_char in replacements:
        if char == rep:
            return new_char
    return char

In [14]:
def clean_text(text: str):
        """Escape the illegal characters for an ontology property"""
        if text is None:
            return None
        # function to escape XML character data
        text = escape(text)
        text = text.replace('\n', '')
        text = text.replace('\r', '')
        text = text.replace('\f', '')
        text = text.replace('\b', '')
        text = text.replace('"', '')
        text = text.replace('[', '')
        text = text.replace(']', '')
        text = text.replace('{', '')
        text = text.replace('}', '')
        text = text.replace('#', '')
        text = text.replace('|', '')
        text = text.replace(' ', '_')
        text = text.replace('σ', '')
        text = text.replace('ℓ', '')
        text = text.replace('Σ', '')
        text = text.replace('ı', '')
        text = text.replace('ē', '')
        text = text.replace('μ', '')
        text = text.replace('Μ', '')
        text = text.replace('Ł', '')
        text = text.replace('ł', '')
        text = text.replace('θ', '')
        text = text.replace('φ', '')
        text = text.replace('Φ', '')
        text = text.replace('̈', '')
        text = text.replace('́', '')
        text = text.replace('′', '')
        text = text.replace('∈', '')
        text = text.replace('ć', '')
        text = text.replace('ź', '')
        text = text.replace('', 'a')
        text = text.replace('⊕','')
        text = text.replace('', 'b')
        text = text.replace('', 'c')
        text = clean_char(text)
        return text

In [15]:
for i in range(len(author_list)):
    author_list[i] = clean_text(author_list[i])

for i in range(len(references_list)):
    references_list[i] = clean_text(references_list[i])

for i in range(len(author_ref_list)):
    author_ref_list[i] = clean_text(author_ref_list[i])

for i in range(len(references_ref_list)):
    references_ref_list[i] = clean_text(references_ref_list[i])


# Création de l'ontologie

In [16]:
from owlready2 import *

In [17]:
# owlready2.JAVA_EXE='C:/Program Files/Common Files/Oracle/Java/javapath/java.exe'
owlready2.JAVA_EXE='/usr/bin/java'

### Création des classes et des propriétés

In [18]:
onto = get_ontology('http://filrouge.org/onto.owl') # nouvelle ontologie, on donne son IRI

with onto:
    # Define an Person
    class Person(Thing):  # nouvelle classe sous-classe de Thing
        pass
    
    # An Person get ideas from another Person
    class gets_ideas_from(Person >> Person): pass


#### Ajout des entités du document dans l'ontologie

In [19]:
with onto:
    for i in range(2):
        globals()[author_list[i]] = Person(name = author_list[i])
        for j in range(2):
            globals()[references_list[j]] = Person(name = references_list[j])
            globals()[author_list[i]].gets_ideas_from.append(globals()[references_list[j]])
    for i in range(2):
        for j in range(2):
            globals()[author_ref_list[j]] = Person(name = author_ref_list[j])
            globals()[author_list[i]].gets_ideas_from.append(globals()[author_ref_list[j]])
    for i in range(2):
            for j in range(2):
                globals()[references_ref_list[j]] = Person(name = references_ref_list[j])
                globals()[author_ref_list[i]].gets_ideas_from.append(globals()[references_ref_list[j]])

In [20]:
with onto:
    AllDifferent([globals()[author_list[0]],globals()[author_ref_list[0]],globals()[references_list[0]],globals()[references_ref_list[0]]])

### Création de règles complexes pour l'ontologie

In [21]:
with onto:
    imp = Imp()
    imp.set_as_rule("Person(?x), Person(?y), Person(?z), \
                     DifferentFrom(?x, ?y), DifferentFrom(?y, ?z), DifferentFrom(?x, ?z),  \
                     gets_ideas_from(?x, ?y), gets_ideas_from(?y, ?z) -> gets_ideas_from(?x, ?z)")

In [22]:
print(len(globals()[author_list[0]].gets_ideas_from))
globals()[author_list[0]].gets_ideas_from

4


[onto.R._J._Weiss, onto.C._Zhang, onto.Ann_Lee, onto.Peng_Jen_Chen]

In [23]:
print(len(globals()[author_ref_list[0]].gets_ideas_from))
globals()[author_ref_list[0]].gets_ideas_from

2


[onto.Pd_Aguero, onto.Jordi_Adell]

In [24]:
with onto:
  print(len(Person.instances()))
  for p in Person.instances(): 
    print(p)

7
onto.Sravya_Popuri
onto.R._J._Weiss
onto.C._Zhang
onto.Peng_Jen_Chen
onto.Ann_Lee
onto.Pd_Aguero
onto.Jordi_Adell


In [25]:
sync_reasoner_pellet([onto], infer_property_values=True, infer_data_property_values=True, debug=True, keep_tmp_file=True)

* Owlready2 * Running Pellet...
    /usr/bin/java -Xmx2000M -cp /usr/local/lib/python3.7/dist-packages/owlready2/pellet/httpclient-4.2.3.jar:/usr/local/lib/python3.7/dist-packages/owlready2/pellet/jena-core-2.10.0.jar:/usr/local/lib/python3.7/dist-packages/owlready2/pellet/aterm-java-1.6.jar:/usr/local/lib/python3.7/dist-packages/owlready2/pellet/commons-codec-1.6.jar:/usr/local/lib/python3.7/dist-packages/owlready2/pellet/jena-iri-0.9.5.jar:/usr/local/lib/python3.7/dist-packages/owlready2/pellet/httpcore-4.2.2.jar:/usr/local/lib/python3.7/dist-packages/owlready2/pellet/jena-tdb-0.10.0.jar:/usr/local/lib/python3.7/dist-packages/owlready2/pellet/jena-arq-2.10.0.jar:/usr/local/lib/python3.7/dist-packages/owlready2/pellet/antlr-3.2.jar:/usr/local/lib/python3.7/dist-packages/owlready2/pellet/pellet-2.3.1.jar:/usr/local/lib/python3.7/dist-packages/owlready2/pellet/xercesImpl-2.10.0.jar:/usr/local/lib/python3.7/dist-packages/owlready2/pellet/slf4j-log4j12-1.6.4.jar:/usr/local/lib/python3.7/d

* Owlready * Adding relation onto.Jordi_Adell gets_ideas_from onto.Ann_Lee
* Owlready * Adding relation onto.Sravya_Popuri gets_ideas_from onto.Pd_Aguero


* Owlready2 * Pellet took 1.4212651252746582 seconds
* Owlready * (NB: only changes on entities loaded in Python are shown, other changes are done but not listed)


In [26]:
print(len(globals()[author_list[0]].gets_ideas_from))
globals()[author_list[0]].gets_ideas_from

5


[onto.R._J._Weiss,
 onto.C._Zhang,
 onto.Ann_Lee,
 onto.Peng_Jen_Chen,
 onto.Pd_Aguero]

In [27]:
onto.save('onto.owl')

In [28]:
onto.destroy('onto.owl')